In [61]:
from imblearn.under_sampling import RandomUnderSampler

In [62]:
rus = RandomUnderSampler()
X_s,y_s = rus.fit_resample(X,y.astype('int'))

In [64]:
X_train,X_test,y_train,y_test = train_test_split(X_s,y_s,test_size=0.3,random_state=42)
ohe = Pipeline([('Encoder',OneHotEncoder(drop = 'first',handle_unknown='ignore'))])
transformer = ColumnTransformer([('One Hot Encoding',ohe,cat)])
model = Pipeline([("Transformer",transformer),
                 ("Estimator",XGBClassifier())])
model.fit(X_train,y_train)
y_pred = model.predict(X_test).astype('object')
y_pred = model.predict(X_test)
print(classification_report(y_test.astype('int'),y_pred))

# ============================================================
# 🚀 SAFE SMOTE (NUMERIC ONLY) + CATEGORICAL MERGE + KMEANS
# ============================================================

from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

print("\n======================")
print("📌 Applying SMOTE on numeric columns only...")
print("======================")

# 1️⃣ Identify numeric + categorical
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

X_num = X[num_cols]
X_cat = X[cat_cols]
y_int = y.astype(int)

# 2️⃣ SMOTE on numeric features
sm = SMOTE(random_state=42)
X_num_sm, y_sm = sm.fit_resample(X_num, y_int)

print("Before SMOTE:", np.bincount(y_int))
print("After  SMOTE:", np.bincount(y_sm))

# ============================================================
# 3️⃣ Match CATEGORICAL rows to SMOTE synthetic samples
# ============================================================

print("\n======================")
print("📌 Assigning categorical values using nearest neighbors...")
print("======================")

# Train nearest neighbors model on original numeric data
nn = NearestNeighbors(n_neighbors=1)
nn.fit(X_num)

# Find nearest real row for each SMOTE row
distances, indices = nn.kneighbors(X_num_sm)

# Assign categorical values based on nearest real sample
X_cat_sm = X_cat.iloc[indices.flatten()].reset_index(drop=True)

# Merge numeric + categorical
X_final = pd.concat([
    pd.DataFrame(X_num_sm, columns=num_cols),
    X_cat_sm
], axis=1)

print("Final merged shape:", X_final.shape)

# ============================================================
# 4️⃣ KMeans on numeric SMOTE data
# ============================================================

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num_sm)

kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

X_final["cluster"] = clusters

print("Cluster counts:", np.bincount(clusters))

# ============================================================
# 5️⃣ Train/test split
# ============================================================

X_train3, X_test3, y_train3, y_test3 = train_test_split(
    X_final, y_sm, test_size=0.3, random_state=42
)

# ============================================================
# 6️⃣ Train XGBoost using your original transformer
# ============================================================

model2 = Pipeline([
    ("Transformer", transformer),   # your original encoder
    ("Estimator", XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
])

model2.fit(X_train3, y_train3)

pred2 = model2.predict(X_test3)

print("\n======================")
print("📊 Report (SMOTE + Categorical Merge + Clustering)")
print("======================")
print(classification_report(y_test3.astype(int), pred2))

C:\Users\Purav Shah\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Purav Shah\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


              precision    recall  f1-score   support

           0       0.96      0.80      0.87      1586
           1       0.82      0.96      0.89      1521

    accuracy                           0.88      3107
   macro avg       0.89      0.88      0.88      3107
weighted avg       0.89      0.88      0.88      3107


📌 Applying SMOTE on numeric columns only...
Before SMOTE: [5073159    5177]
After  SMOTE: [5073159 5073159]

📌 Assigning categorical values using nearest neighbors...
Final merged shape: (10146318, 10)
Cluster counts: [10144824     1494]


C:\Users\Purav Shah\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [15:45:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Purav Shah\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



📊 Report (SMOTE + Categorical Merge + Clustering)
              precision    recall  f1-score   support

           0       0.60      0.72      0.65   1522277
           1       0.65      0.52      0.58   1521619

    accuracy                           0.62   3043896
   macro avg       0.62      0.62      0.62   3043896
weighted avg       0.62      0.62      0.62   3043896

